<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction

In this lab, we convert addresses into their equivalent latitude and longitude values and use the Foursquare API to explore neighborhoods in Toronto. Using, the **explore** function we get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters using the *k*-means clustering algorithm to complete this task. Finally, the Folium library is used to visualize the neighborhoods in Toronto and their emerging clusters.

## Importing required libraries

In [ ]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')